# HOW TO USE THESE SCRIPTS

## SETTING UP THE ENVIRONMENT
First of all STEAD and INSTANCE dataset must be downloaded. Then STEAD dataset will involve a csv file and a hdf5 file while INSTANCE dataset would have seperate files for noise and earthquake waveforms. After downloading properly, config file (config.py) must be modified accordingly. After proper configuration, datasets must be preprocessed in order to optimize training and testing procedures. In the optimization process each dataset is split into chunks which will be grouped for kfold training and testing purposes afterwards. "KFoldEnvironment" class governs forming proper splits and chunks. At the end, some of the chunks will be used for training, some of them for validation and testing. This will be done for each split. 


## TRAINING MODELS
After preprocessed datasets are generated (it can take upto 2-3hours per dataset), training can be done. "Trainer" class governs training processes. Training can only be done split based(although testing is rather flexible, for training at this version of the software, all of the chunks included in the training set of split must be used). Training is done at the models in "detector_models.py" file since models are split to two classes. Please note that training is done compatible with Kfold validation. Trainer itself generates a kfold environment to be used for training implicitly.

In [ ]:
from detector_models import AutocovarianceDetector30s
from trainer import Trainer

# Create a trainer object.
trainer = Trainer(
    model_ctor=AutocovarianceDetector30s,
)

# Train the model on INSTANCE dataset with learning rate 1e-4 and epsilon 1e-7. Training is done on folds 0, 1, 2, 3, 4.
trainer.train(
    "instance",
    epochs=1,
    learning_rate=1e-4,
    epsilon=1e-7,
    splits=[0, 1],
)

## TESTING MODELS
Testing models encapsulates monitoring the hidden layers or some variables of the model rather then running classification algorithm directly. Classification is done by using the variables stored afterwards. Models are composed of two different classes, "detector" and "monitor" models namely. Detector models are the models which can be trained while monitor models are used to monitor different variables for the application of some methods. As a result, monitor models are not actually models in a classical sense; they are just constructs used for classification purposes.

In order to test models, two different kfold environment should be formed. One of them will be the training environment.

In [ ]:
from detector_models import AutocovarianceDetector30s
from monitor_models import AutocovarianceMonitor30s
from kfold_environment import KFoldEnvironment
from tester import KFoldTester

# Form train environment. 
train_kenv = KFoldEnvironment(
    dataset="instance",
        subsampling_factor=1.0,
    )

# Form test environment.
test_kenv = KFoldEnvironment(
    dataset="stead",
    subsampling_factor=1.0,
)

# Create a tester object. Constructors from detector and monitor models should be specified.
tester = KFoldTester(
    train_environment=train_kenv,
    test_environment=test_kenv,
    detector_ctor=AutocovarianceDetector30s,
    monitor_ctor=AutocovarianceMonitor30s,
)

# Test the model on folds 0. Monitoring results will be stored on 
# {HOME_DIR}/results{exp_name}/training_{train_dataset}/test_{test_dataset}/{monitor_model_name}/split_{split_number}/epoch{epoch}_monitoredparams{monitor_params}.hdf5
# Besides at the same directory traces.csv file is generated. This file contains the traces of the monitored parameters in the same order with the hdf5 file. 
tester.test(
    exp_name="autocovariance_detector_30s",
    splits=[0],
    monitor_params=["fcov"],
    method_params={"epoch": 19},
)